In [1]:
import os
import json
import requests

In [2]:
GROQ_API_KEY = os.getenv("GROQ_API_KEY", "REPLACE_WITH_YOUR_KEY")
GROQ_API_URL = "https://api.groq.com/v1/chat/completions"

HEADERS = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {GROQ_API_KEY}"
}


In [3]:
EXTRACTION_SCHEMA = {
    "type": "object",
    "properties": {
        "name": {"type": ["string", "null"]},
        "email": {"type": ["string", "null"]},
        "phone": {"type": ["string", "null"]},
        "location": {"type": ["string", "null"]},
        "age": {"type": ["integer", "null"]}
    },
    "required": []
}

In [4]:
def validate_extraction(obj: dict):
    errors = []
    if not isinstance(obj, dict):
        return False, ["Not a JSON object"]

    if "name" in obj and obj["name"] not in [None, ""] and not isinstance(obj["name"], str):
        errors.append("name must be string or null")

    if "email" in obj and obj["email"] not in [None, ""] and not isinstance(obj["email"], str):
        errors.append("email must be string or null")

    if "phone" in obj and obj["phone"] not in [None, ""] and not isinstance(obj["phone"], str):
        errors.append("phone must be string or null")

    if "location" in obj and obj["location"] not in [None, ""] and not isinstance(obj["location"], str):
        errors.append("location must be string or null")

    if "age" in obj and obj["age"] not in [None, ""]:
        if not isinstance(obj["age"], int):
            errors.append("age must be integer or null")

    return len(errors) == 0, errors


In [5]:
def call_model_for_extraction(text: str) -> dict:
    system_prompt = (
        "You are a JSON extractor. Extract the fields name, email, phone, location, and age "
        "from the given chat. Return only a JSON object that matches this schema: \n"
        + json.dumps(EXTRACTION_SCHEMA)
        + "\nIf a field is missing, set it to null."
    )

    body = {
        "model": "gpt-4o-mini",   # Example model, replace with Groq-supported model if needed
        "messages": [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": f"Chat message:\n{text}\n\nReturn only the JSON object."}
        ],
        "temperature": 0,
        "max_tokens": 300
    }

    resp = requests.post(GROQ_API_URL, headers=HEADERS, data=json.dumps(body))
    if resp.status_code != 200:
        print("Error:", resp.status_code, resp.text)
        return {}

    data = resp.json()
    assistant_text = data["choices"][0]["message"]["content"]

    try:
        parsed = json.loads(assistant_text)
        return parsed
    except Exception as e:
        print("Could not parse JSON:", e)
        print("Raw output:", assistant_text)
        return {}


In [6]:
sample_chats = [
    "Hello, I'm Ravi Kumar. My email is ravi.k@example.com and my phone is +91-9876543210. I'm 22 and located in Pune.",
    "Contact: Priya; priya123@domain.org; phone 07000111222; lives in Chennai.",
    "Hey, this is Sam. I'd rather not share my phone. Age 30. Email: sam30@mail.com"
]

for chat in sample_chats:
    print("\nChat:", chat)
    extracted = call_model_for_extraction(chat)
    print("Extracted:", extracted)
    ok, errs = validate_extraction(extracted)
    print("Valid?", ok)
    if not ok:
        print("Errors:", errs)


Chat: Hello, I'm Ravi Kumar. My email is ravi.k@example.com and my phone is +91-9876543210. I'm 22 and located in Pune.
Error: 404 {"error":{"message":"Unknown request URL: POST /v1/chat/completions. Please check the URL for typos, or see the docs at https://console.groq.com/docs/","type":"invalid_request_error","code":"unknown_url"}}

Extracted: {}
Valid? True

Chat: Contact: Priya; priya123@domain.org; phone 07000111222; lives in Chennai.
Error: 404 {"error":{"message":"Unknown request URL: POST /v1/chat/completions. Please check the URL for typos, or see the docs at https://console.groq.com/docs/","type":"invalid_request_error","code":"unknown_url"}}

Extracted: {}
Valid? True

Chat: Hey, this is Sam. I'd rather not share my phone. Age 30. Email: sam30@mail.com
Error: 404 {"error":{"message":"Unknown request URL: POST /v1/chat/completions. Please check the URL for typos, or see the docs at https://console.groq.com/docs/","type":"invalid_request_error","code":"unknown_url"}}

Extrac